In [ ]:
import numpy as np
import pandas

In [ ]:
# Cleaning Data

df = pandas.read_csv('train.csv', encoding='latin-1', names=["Sentiment", "ID", "Date", "Query","Username","Text"])
df = df.drop(["ID", "Date", "Query","Username"], axis=1)
df['Text'] = df['Text'].str.replace("&gt;_&lt;", "sad")
df['Text'] = df['Text'].str.replace(";-;", "sad")
df['Text'] = df['Text'].str.replace(";_;", "sad")
df['Text'] = df['Text'].str.replace(" D: ", " sad ")
df['Text'] = df['Text'].str.replace(" :D ", " happy ")
df['Text'] = df['Text'].str.replace(" ;D", " happy")
df['Text'] = df['Text'].str.replace("xd", "laugh")
df['Text'] = df['Text'].str.replace("=.=", "confused")
df['Text'] = df['Text'].str.replace("o_o", "confused")
df['Text'] = df['Text'].str.replace("O_o", "confused")
df['Text'] = df['Text'].str.lower()
df["Text"] = df["Text"].str.replace('(\@\w+.*?)',"")
df["Text"] = df["Text"].str.replace('(\#\w+.*?)',"")
df['Text'] = df['Text'].str.replace('http[^\s]*',"")
df['Text'] = df['Text'].str.replace(".", "")
df['Text'] = df['Text'].str.replace(",", "")
df['Text'] = df['Text'].str.replace("-", "")
df['Text'] = df['Text'].str.replace("`", "")
df['Text'] = df['Text'].str.replace("'", "")
df['Text'] = df['Text'].str.replace(": ", " ")
df['Text'] = df['Text'].str.replace("; ", " ")
df['Text'] = df['Text'].str.replace("&amp;", "and")
df['Text'] = df['Text'].str.replace("&quot;", "and")
df['Text'] = df['Text'].str.replace("?", "")
df['Text'] = df['Text'].str.replace("!", "")
df['Text'] = df['Text'].str.replace("*", "")
df["Text"] = df["Text"].str.replace('ï¿½',"")
df['Text'] = df['Text'].str.replace("&amp", "")
df['Text'] = df['Text'].str.replace("&quot", "")
df['Text'] = df['Text'].str.replace("&gt", "")
df['Text'] = df['Text'].str.replace("&lt", "")
df['Text'] = df['Text'].str.replace("  ", " ")
df['Text'] = df['Text'].str.replace("  ", " ")
df['Text'] = df['Text'].str.replace("  ", " ")
df['Text'] = df['Text'].str.replace(" @ ", " ")
df['Text'] = df['Text'].str.replace("upits", "up its")
df['Text'] = df['Text'].str.replace("Of to ", "off to ")
df['Text'] = df['Text'].str.replace("brightside", "bright side")
df['Text'] = df['Text'].str.replace("thaanks", "thanks")
df['Text'] = df['Text'].str.replace("thaaanks", "thanks")
df['Text'] = df['Text'].str.lstrip(' ')
df['Text'] = df['Text'].str.rstrip(' ')

In [ ]:
# Dataset

import torch
import numpy as np
from transformers import BertTokenizer

MAXLENGTH = 512


tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):

        self.labels = [labels for labels in df['Sentiment']]
        self.texts = [tokenizer(text, padding='max_length', max_length = MAXLENGTH, truncation=True, return_tensors="pt") for text in df['Text']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)
        return batch_texts, batch_y

In [ ]:
np.random.seed(42)
trainX, evaluationX, testX = np.split(df.sample(frac=1, random_state=42), [int(.8*len(df)), int(.9*len(df))])

In [ ]:
from pathlib import Path

# 1. Create models directory 
MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True, exist_ok=True)

# 2. Create model save path 
MODEL_NAME = "model.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

# 3. Save the model state dict 
#torch.save(obj=model.state_dict(),f=MODEL_SAVE_PATH)

In [ ]:
from torch import nn
from transformers import BertModel

class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 5) 
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer

In [ ]:
from torch.optim import Adam
from tqdm import tqdm

def train(model, train_data, val_data, learning_rate, epochs):

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)
                
                batch_loss = criterion(output, train_label.long())
                total_loss_train += batch_loss.item()
                
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label.long())
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
                | Train Accuracy: {total_acc_train / len(train_data): .3f} \
                | Val Loss: {total_loss_val / len(val_data): .3f} \
                | Val Accuracy: {total_acc_val / len(val_data): .3f}')
            MODEL_NAME = "model" + str(epoch_num + 1) +".pth"
            torch.save(obj=model.state_dict(),f=MODEL_PATH / MODEL_NAME)
                
                  
EPOCHS = 5 # ---------------
model = BertClassifier()
LR = 1e-6